In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from time import time
import matplotlib.pyplot as plt

# This is function for project1

## 1) Function with white markdown is subfunction
## 2) Function with blue markdown is aggregate function from subfunction ([You only need to use function with blue markdown])

### show_head_text is function for read txt file and show head -n line

In [2]:
def show_head_text(fileName, n_line):

    f = open(fileName, "r")
    for i in range(n_line):
        print(f.readline())
    f.close()

### convert text file to dataframe without any edit format

In [3]:
def txt_to_df(fileName):

    f = open(fileName,"r")

    ### count for determine line
    count = 0

    # metadata_df = pd.DataFrame(columns = header)
    info_data = dict()
    for line in f:
        count += 1
        tmp = line

        if count < 10:
            head = tmp[2:].split(":")[0]
            data = tmp[2:].split(":")[1][1:-1]
            info_data[head] = [data]

        ### This line we determine that is header
        elif count == 10:

            ### We remove "% " at first 2 string and split each string 
            header = tmp[2:].split(", ")

            ### The last word will contains "\n" so we remove it
            header[-1] = header[-1][:-1]
            
            ### Create dataframe to support data will be coming after this line
            df = pd.DataFrame(columns = header)

        else:
            data = tmp.split()
            data_df = pd.DataFrame([data], columns = header)
            df = df.append(data_df)
    
    f.close()
    
    df = df.reset_index().drop(columns = ["index"])
    info_df = pd.DataFrame.from_dict(info_data)
    return info_df, df

### convert any time unit with len one eg. 7 => 07

In [35]:
def make_len2(time):
    if len(time) == 1:
        time = "0" + time
    return time

### convert all specfic headers to len 2 by use make_len2

In [36]:
def all_make_len2(df, headers):

    for header in headers:  
        df[header] = df[header].apply(lambda x: make_len2(x))
    return df

### create datetime columns by input specific (eg. hour, month ,day, utc hour)

In [ ]:
def create_datetime(df, headers, string_format):
    df["datestr"] = ""
    for header in headers:
        df["datestr"] += df[header]
    df["datetime"] = pd.to_datetime(df["datestr"], format=string_format)
    return df

### Shift datetime to specific utc

In [ ]:
def shiftTime(hour):
    df["datetime"] = df["datetime"] + timedelta(hours=hour)
    df["UTC Hour"] = df["UTC Hour"] + hour
    return df["datetime"]

## [Use this function to convert txt file to dataframe with datetime]

In [ ]:
def txt_to_df_with_shift_datetime(fileName):

    info_df, df = txt_to_df(fileName)
    header_to_numeric = ["PM2.5", "PM10_mask", "Retrospective"]
    for header in header_to_numeric:
        df[header] = pd.to_numeric(df[header])
    df = all_make_len2(df, ["Month", "Day", "UTC Hour"])
    df = create_datetime(df, ["Year","Month","Day","UTC Hour"], "%Y%m%d%H")
    df.drop(columns = ["datestr"], inplace = True)
    df["datetime"] = df["datetime"] + timedelta(hours=7)
    
    return info_df, df

### check datetime value is out of range or not

In [ ]:
def check_city_feature_unique(df, headers, city):
    print("-"*30)
    for header in headers:
        print(city + " " + header)
        print(df[header].unique())
        print("-"*30)

### Check null value all columns in dataframe

In [ ]:
def check_null_all(df):
    null_counts = df.isnull().sum()
    print("Number of null values in each column:\n{}".format(null_counts))

### Extract date from datetime(string type) => Wind dataframe

In [ ]:
def extract_date_string(df):    
    df["Year"] = df["datetime"].apply(lambda x: int(x[:4]))
    df["Month"] = df["datetime"].apply(lambda x: int(x[5:7]))
    df["Day"] = df["datetime"].apply(lambda x: int(x[8:10]))
    df["UTC Hour"] = df["datetime"].apply(lambda x: int(x[11:13]))
    return df